<a href="https://colab.research.google.com/github/Deepika4C69/Bytexl/blob/main/AI_agent_for_online_meeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub openai-whisper transformers langdetect googletrans==4.0.0-rc1 nltk
!apt install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.1 MB/s eta 0:00:00


In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_file("/content/meeting audio.mp3", format="mp3")
audio.export("/content/meeting audio.wav", format="wav")


In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("/content/meeting audio.wav")
transcript = result["text"]

print(transcript)


In [ ]:
from transformers import pipeline
import torch

summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)
max_transcript_length = 1000
if len(transcript) > max_transcript_length:
    transcript = transcript[:max_transcript_length]
    print(f"Transcript truncated to {max_transcript_length} characters.")
if not transcript.strip():
    summary = "Transcript is empty or too short to summarize."
else:
    summary = summarizer(transcript, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    print("Summary:", summary)
print("Summary:", summary)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Transcript truncated to 1000 characters.
Summary: The arm of Springfield acknowledges that we are gathered on ancestral lands. Treaty one territory, traditional territory of the Anishinaabe, Kri, Oji K.
Summary: The arm of Springfield acknowledges that we are gathered on ancestral lands. Treaty one territory, traditional territory of the Anishinaabe, Kri, Oji K.


In [ ]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")
sentiment = sentiment_pipeline(transcript)
print("Overall Sentiment:", sentiment)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


Overall Sentiment: [{'label': 'POSITIVE', 'score': 0.9992223978042603}]


In [ ]:
qa_pipeline = pipeline("question-answering")

question = "What was discussed in the meeting?"
qa_result = qa_pipeline(question=question, context=summary)
print("Q&A Result:", qa_result)


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


Q&A Result: {'score': 0.2512718141078949, 'start': 77, 'end': 97, 'answer': 'Treaty one territory'}


In [ ]:
from googletrans import Translator

translator = Translator()
translated = translator.translate(summary, dest='fr')  # 'fr' for French
print("Translated Summary:", translated.text)


Translated Summary: Le bras de Springfield reconnaît que nous sommes rassemblés sur des terres ancestrales.Traité One Territory, Territoire traditionnel de l'Anishinaabe, Kri, Oji K.


In [ ]:
import smtplib
from email.message import EmailMessage

In [ ]:
SENDER_EMAIL = "poojitha307.vintha@gmail.com"
APP_PASSWORD = "gvfz uitg gduh sskb"

In [ ]:
def send_email(summary, recipient):
    msg = EmailMessage()
    msg.set_content(summary)
    msg["Subject"] = "Meeting Summary"
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(SENDER_EMAIL, APP_PASSWORD)
            smtp.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", e)

In [ ]:
summary_text = "Le bras de Springfield reconnaît que nous sommes rassemblés sur des terres ancestrales.Traité One Territory, Territoire traditionnel de l'Anishinaabe, Kri, Oji K."
recipient_email = "poojitha307.vintha@gmail.com"
send_email(summary_text, recipient_email)

Email sent successfully!
